In [2]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [5]:
# cuz i installed imblearn in the 3.9 folder
# but i usually launch jupyter lab from the 3.8 folder
## if 3.8, relaunch using: python3 -m notebook
from platform import python_version
print(python_version())

3.9.1


In [6]:
# loading data files

# has clean review text
dataset = pd.read_json("dramainfo_revclean.json")

# tfidf
tfidf_rev = np.load("tfidfvec.npy")

# word2vec
w2v_rev = np.load("w2featvec.npy")

In [57]:
# no need for adjustment
'''
political_drama
drama
'''

# undersample 1, oversample 0
'''
melodrama_romance
romance
'''

# undersample 0, oversample 1
'''
sitcom_comedy
comedy
mystery
youth_school
fantasy_supernatural_horror
family
war_historical
historical
friendship
life
action
thriller
'''

dataset.iloc[:,16:-1].mean().sort_values(ascending=False)

melodrama_romance              0.695324
romance                        0.684211
political_drama                0.525477
drama                          0.513315
sitcom_comedy                  0.369469
comedy                         0.368421
mystery                        0.153282
youth_school                   0.150346
fantasy_supernatural_horror    0.146152
family                         0.143007
war_historical                 0.132103
historical                     0.130216
friendship                     0.114280
life                           0.114070
action                         0.105892
thriller                       0.102118
dtype: float64

In [7]:
import imblearn

In [8]:
# !!!!!!!!!!

In [9]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbPipeline

In [184]:
Xtfidf_train, Xtfidf_test, ytfidf_train, ytfidf_test = train_test_split(tfidf_rev,
                                                                        dataset.action,
                                                                        test_size=0.3,
                                                                        random_state=18)

In [187]:
target_counts = Counter(ytfidf_train)
print(target_counts)

Counter({0: 2988, 1: 350})


In [16]:
def over_under_strat(minority,majority):
    strat_lst = []
    
    over_ratio = np.round(minority/majority,2) + 0.1
    
    while over_ratio < 0.8:
        strat_lst.append((over_ratio, 0.8))
        over_ratio = over_ratio + 0.1
    
    strat_lst.append((0.8,0.8))
    
    return strat_lst

In [236]:
strat_lst = over_under_strat(target_counts[1],target_counts[0])
strat_lst

[(0.22, 0.8),
 (0.32, 0.8),
 (0.42000000000000004, 0.8),
 (0.52, 0.8),
 (0.62, 0.8),
 (0.72, 0.8),
 (0.8, 0.8)]

In [18]:
models = [("logreg", LogisticRegression(max_iter=200)), 
          ("rf", RandomForestClassifier()),
          ("gnb", GaussianNB())]

scoring = ["accuracy",
           "balanced_accuracy",
           "precision",
           "recall",
           "f1",
           "roc_auc"]

In [19]:
def run_pipeline(steps, X_train, y_train, X_test, scoring=scoring):
    pipeline = imbPipeline(steps=steps)
    
    cv = model_selection.RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
    scores = model_selection.cross_validate(pipeline, X_train, y_train,
                                            scoring=scoring, cv=cv, n_jobs=-1)
    
    clf = pipeline.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    return scores, y_pred

In [295]:
result_dfs = []
pred_list = []

for name, model in models:
    for over_strat, under_strat in strat_lst:
        print("Running ... {:<9}{:<.2}".format(name,over_strat))
        steps = [("over", RandomOverSampler(sampling_strategy=over_strat)),
                 ("under",RandomUnderSampler(sampling_strategy=under_strat)),
                 (name, model)]
        result,pred = run_pipeline(steps,
                                   Xtfidf_train,ytfidf_train,
                                   Xtfidf_test)
        model_df = pd.DataFrame(result)
        model_df["model"] = name
        model_df["oversample_strat"] = over_strat
        result_dfs.append(model_df)

        pred_list.append(pred)

Running ... logreg   0.22
Running ... logreg   0.32
Running ... logreg   0.42
Running ... logreg   0.52
Running ... logreg   0.62
Running ... logreg   0.72
Running ... logreg   0.8
Running ... rf       0.22
Running ... rf       0.32
Running ... rf       0.42


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Running ... rf       0.52


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Running ... rf       0.62


...

Running ... rf       0.72


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Fram

Running ... rf       0.8


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Fram

Running ... gnb      0.22
Running ... gnb      0.32
Running ... gnb      0.42
Running ... gnb      0.52
Running ... gnb      0.62
Running ... gnb      0.72
Running ... gnb      0.8


In [296]:
results = pd.concat(result_dfs, ignore_index=True)

In [318]:
results_grp = results.groupby(["model","oversample_strat"]).agg([np.mean,np.std])
results_grp

fit_time             score_time            \
                               mean         std       mean       std   
model  oversample_strat                                                
gnb    0.22               21.110729    9.441018   2.209390  0.525522   
       0.32               27.945038   12.150217   2.131880  0.593280   
       0.42               33.452614   15.091641   2.034965  0.565789   
       0.52               36.129575   15.864311   2.278499  0.661459   
       0.62               45.158232   20.026900   2.015513  0.564236   
       0.72               77.374633   36.267202   1.803522  0.576432   
       0.80               79.853122   36.810557   2.407952  0.870287   
logreg 0.22               20.224800    8.147673   0.243940  0.099017   
       0.32               27.162181   10.396669   0.313554  0.171240   
       0.42               35.822748   11.990034   0.348154  0.212228   
       0.52              152.861339   58.206517   0.413579  0.247714   
       0.62              201.801743   88.304647   0.369299  0.213062   
       0.72              298.382141  123.020734   0.412089  0.210972   
       0.80              358.215264  158.342680   0.380136  0.192550   
rf     0.22               25.105655   10.245880   0.469073  0.168074   
       0.32               31.657221   12.670421   0.493433  0.149439   
       0.42               40.220099   16.184407   0.543528  0.145720   
       0.52               50.233886   20.149587   0.536880  0.174907   
       0.62               54.561446   20.999399   0.509527  0.154131   
       0.72               81.002549   33.300063   0.553219  0.227000   
       0.80              127.798042   55.629144   0.786164  0.522189   

                        test_accuracy           test_balanced_accuracy  \
                                 mean       std                   mean   
model  oversample_strat                                                  
gnb    0.22                  0.789097  0.010631               0.580762   
       0.32                  0.829840  0.009449               0.558744   
       0.42                  0.847811  0.008540               0.544820   
       0.52                  0.855899  0.006999               0.536094   
       0.62                  0.862490  0.006291               0.534100   
       0.72                  0.865485  0.006226               0.531359   
       0.80                  0.867131  0.008380               0.528495   
logreg 0.22                  0.895745  0.007230               0.695827   
       0.32                  0.899339  0.007791               0.697203   
       0.42                  0.899337  0.008403               0.699094   
       0.52                  0.902483  0.009150               0.700221   
       0.62                  0.903383  0.009269               0.695679   
       0.72                  0.904880  0.008004               0.695885   
       0.80                  0.905927  0.009006               0.698361   
rf     0.22                  0.898592  0.005988               0.549221   
       0.32                  0.897094  0.003407               0.518114   
       0.42                  0.896045  0.001795               0.506808   
       0.52                  0.895895  0.001796               0.504833   
       0.62                  0.895297  0.000804               0.501345   
       0.72                  0.895296  0.000502               0.500714   
       0.80                  0.895147  0.000081               0.500000   

                                  test_precision           test_recall  \
                              std           mean       std        mean   
model  oversample_strat                                                  
gnb    0.22              0.018094       0.192508  0.017216    0.317143   
       0.32              0.021848       0.204089  0.032902    0.215714   
       0.42              0.018407       0.207493  0.034742    0.161429   
       0.52              0.018616       0.204951  0.046745    0.131429   
   

In [335]:
for i in range(len(pred_list)):
    name = models[int(i/len(strat_lst))][0]
    oversamp = strat_lst[i%7][0]
    
    print("\n{:<9}{:<.2}".format(name,oversamp))
    print(classification_report(ytfidf_test, pred_list[i]))


logreg   0.22
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      1276
           1       0.47      0.48      0.47       155

    accuracy                           0.89      1431
   macro avg       0.70      0.71      0.71      1431
weighted avg       0.89      0.89      0.89      1431


logreg   0.32
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1276
           1       0.50      0.48      0.49       155

    accuracy                           0.89      1431
   macro avg       0.72      0.71      0.71      1431
weighted avg       0.89      0.89      0.89      1431


logreg   0.42
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1276
           1       0.47      0.46      0.47       155

    accuracy                           0.89      1431
   macro avg       0.70      0.70      0.70      1431
weighted avg       0.89      

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [336]:
# logistic regression seems to do the best
# should upsample to at least around 0.4

In [10]:
# now for w2v

In [11]:
Xw2v_train, Xw2v_test, yw2v_train, yw2v_test = train_test_split(w2v_rev,
                                                                dataset.action,
                                                                test_size=0.3,
                                                                random_state=18)

In [14]:
target_counts2 = Counter(yw2v_train)
print(target_counts2)

Counter({0: 2988, 1: 350})


In [17]:
strat_lst2 = over_under_strat(target_counts2[1],target_counts2[0])
strat_lst2

[(0.22, 0.8),
 (0.32, 0.8),
 (0.42000000000000004, 0.8),
 (0.52, 0.8),
 (0.62, 0.8),
 (0.72, 0.8),
 (0.8, 0.8)]

In [20]:
result_dfs2 = []
pred_list2 = []

for name, model in models:
    for over_strat, under_strat in strat_lst2:
        print("Running ... {:<9}{:<.2}".format(name,over_strat))
        steps = [("over", RandomOverSampler(sampling_strategy=over_strat)),
                 ("under",RandomUnderSampler(sampling_strategy=under_strat)),
                 (name, model)]
        result,pred = run_pipeline(steps,
                                   Xw2v_train,yw2v_train,
                                   Xw2v_test)
        model_df = pd.DataFrame(result)
        model_df["model"] = name
        model_df["oversample_strat"] = over_strat
        result_dfs2.append(model_df)

        pred_list2.append(pred)

Running ... logreg   0.22
Running ... logreg   0.32
Running ... logreg   0.42
Running ... logreg   0.52
Running ... logreg   0.62
Running ... logreg   0.72
Running ... logreg   0.8
Running ... rf       0.22
Running ... rf       0.32
Running ... rf       0.42
Running ... rf       0.52
Running ... rf       0.62
Running ... rf       0.72


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Running ... rf       0.8


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Running ... gnb      0.22
Running ... gnb      0.32
Running ... gnb      0.42
Running ... gnb      0.52
Running ... gnb      0.62
Running ... gnb      0.72
Running ... gnb      0.8


In [21]:
results2 = pd.concat(result_dfs2, ignore_index=True)

In [22]:
results_grp2 = results2.groupby(["model","oversample_strat"]).agg([np.mean,np.std])
results_grp2

fit_time           score_time            \
                             mean       std       mean       std   
model  oversample_strat                                            
gnb    0.22              0.006684  0.001502   0.005805  0.002440   
       0.32              0.010896  0.003072   0.008961  0.004188   
       0.42              0.010231  0.004272   0.006271  0.001987   
       0.52              0.011836  0.003489   0.007639  0.002551   
       0.62              0.012750  0.005084   0.007163  0.002667   
       0.72              0.013662  0.004214   0.006118  0.001581   
       0.80              0.012003  0.002696   0.005860  0.001809   
logreg 0.22              0.024860  0.007985   0.005538  0.002836   
       0.32              0.027563  0.006714   0.003403  0.000130   
       0.42              0.033913  0.011346   0.003464  0.000181   
       0.52              0.045337  0.009021   0.004429  0.001857   
       0.62              0.064383  0.017667   0.005211  0.002976   
       0.72              0.084620  0.023317   0.005686  0.004076   
       0.80              0.100464  0.033036   0.004950  0.003059   
rf     0.22              0.660052  0.090416   0.034865  0.006583   
       0.32              0.921280  0.119706   0.036083  0.006035   
       0.42              1.181492  0.155643   0.034284  0.007452   
       0.52              1.456925  0.201010   0.036746  0.008508   
       0.62              1.763797  0.261399   0.036721  0.009146   
       0.72              2.043051  0.310314   0.035242  0.008151   
       0.80              2.243885  0.309508   0.036165  0.008475   

                        test_accuracy           test_balanced_accuracy  \
                                 mean       std                   mean   
model  oversample_strat                                                  
gnb    0.22                  0.489068  0.127958               0.518489   
       0.32                  0.390204  0.071543               0.510562   
       0.42                  0.400682  0.079361               0.517044   
       0.52                  0.415069  0.051363               0.527604   
       0.62                  0.444431  0.096333               0.537698   
       0.72                  0.404280  0.066049               0.526622   
       0.80                  0.429597  0.078433               0.538871   
logreg 0.22                  0.849758  0.012463               0.514376   
       0.32                  0.817859  0.014262               0.521154   
       0.42                  0.814111  0.011198               0.522213   
       0.52                  0.808272  0.009841               0.535347   
       0.62                  0.796887  0.011286               0.527096   
       0.72                  0.797188  0.009569               0.536723   
       0.80                  0.790147  0.011982               0.533421   
rf     0.22                  0.817108  0.018393               0.530192   
       0.32                  0.867737  0.008146               0.514959   
       0.42                  0.883764  0.004159               0.517605   
       0.52                  0.890353  0.002809               0.509305   
       0.62                  0.893349  0.002730               0.507825   
       0.72                  0.893948  0.001699               0.503745   
       0.80                  0.894997  0.001449               0.504961   

                                  test_precision           test_recall  \
                              std           mean       std        mean   
model  oversample_strat                                                  
gnb    0.22              0.026849       0.113318  0.010184    0.555714   
       0.32              0.025109       0.108483  0.008449    0.662857   
       0.42              0.041681       0.110831  0.013154    0.664286   
       0.52              0.048198       0.113676  0.014604    0.670000   
       0.62              0.035075       0.118627  0.014798    0.655714   
       0.72            

In [23]:
for i in range(len(pred_list2)):
    name = models[int(i/len(strat_lst2))][0]
    oversamp = strat_lst2[i%7][0]
    
    print("\n{:<9}{:<.2}".format(name,oversamp))
    print(classification_report(yw2v_test, pred_list2[i]))


logreg   0.22
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1276
           1       0.15      0.12      0.14       155

    accuracy                           0.83      1431
   macro avg       0.53      0.52      0.52      1431
weighted avg       0.82      0.83      0.82      1431


logreg   0.32
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      1276
           1       0.16      0.15      0.15       155

    accuracy                           0.82      1431
   macro avg       0.53      0.53      0.53      1431
weighted avg       0.82      0.82      0.82      1431


logreg   0.42
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      1276
           1       0.16      0.17      0.17       155

    accuracy                           0.81      1431
   macro avg       0.53      0.53      0.53      1431
weighted avg       0.82      

In [24]:
# looks like it's:
## tfidf
## logistic regression
## oversample to ratio 0.4-0.6